In [7]:
import numpy as np

# 判断是否存在奇点
def jidian(G_set, n):
    jidian_list = []
    for ii in range(n):
        if sum(G_set[ii] == inf) % 2 == 1:
            jidian_list.append(ii)
    return jidian_list


In [8]:
# 求奇点间的最短链，用Floyd算法
def floyd(w, n, point_set):
    # 最短路的权矩阵u(n * n) 和各顶点的后进标号矩阵r(n * n)
    u = np.zeros((n, n))
    r = np.full((n, n), 0, dtype=int)

    for k in range(n + 1):
        # 初始化，弧数不多于1时，最短路即为两顶点的弧的权，从权矩阵中提取
        if k == 0:
            u = w
            # 记录第一条弧的头
            for i in range(n):
                for j in range(n):
                    r[i][j] = j + 1

        else:
            for i in range(n):
                for j in range(n):
                    if u[i][k - 1] + u[k - 1][i] < 0:
                        print('存在负回路！Floyd算法不适用！')
                        break
                    else:
                        if u[i][j] > u[i][k - 1] + u[k - 1][j]:
                            temp_r = r
                            r[i][j] = temp_r[i][k - 1]
                            temp_u = u
                            u[i][j] = temp_u[i][k - 1] + temp_u[k - 1][j]

    shortest_path_set = {}
    shortest_weight_set = {}
    # 正向追踪路径
    for i in point_set:
        for j in point_set:
            if j > i:
                # 将起点加入路径
                vertex = [i]
                if i != j:
                    temp = r[i][j]
                    while temp != j + 1:
                        vertex.append(temp - 1)
                        temp = r[temp - 1][j]
                    else:
                        vertex.append(temp - 1)
                    w_sum = 0
                # 求每条最短路径的权和
                for kk in range(len(vertex) - 1):
                    k1 = vertex[kk]
                    k2 = vertex[kk + 1]
                    w_sum += w[k1][k2]
                shortest_path_set[(i, j)] = vertex
                shortest_weight_set[(i, j)] = int(w_sum)
    return shortest_path_set, shortest_weight_set


In [9]:
# 求最小权匹配
def min_match(shortest_weight_set, length):
    path_set = list(shortest_weight_set.keys())
    shortest_match_set = []
    shortest_match_weight = 100
    for i in range(length - 1):
        match_set = [path_set[i]]
        path_weight = shortest_weight_set[path_set[i]]
        for path in path_set:
            count = 0
            for match in match_set:
                if path[0] not in match and path[1] not in match:
                    count += 1
            if count == len(match_set):
                match_set.append(path)
                path_weight += shortest_weight_set[path]
        if path_weight < shortest_match_weight:
            shortest_match_set = match_set
            shortest_match_weight = path_weight
    return shortest_match_set

In [10]:
# 欧拉回路求解
def euler(matrix, first_point):
    first = first_point
    path = [first]
    # 判断矩阵行中是否全为零

    flag = 0
    while flag == 0:
        not_zero_set = np.where(matrix[first] != 0)[0]
        # 若矩阵行中不全为零
        # 取矩阵行中首一非零元素的位置，若走不通再取下一个位置
        locate = 0
        if not_zero_set.size > 0:
            second = not_zero_set[locate]
            if np.sum(matrix[second]) == 1:
                if np.sum(matrix) == 2 and second == first_point:
                    flag = 1
                else:
                    locate += 1
                    second = not_zero_set[locate]

            # print(matrix[second])
            while ~np.any(matrix[second]):
                locate += 1
                second = not_zero_set[locate]

            # 由于矩阵是对称的，将对应位置的标记均减一
            matrix[first][second] -= 1
            matrix[second][first] -= 1
            path.append(second)
            # print(path)
            # print(matrix)
            first = second
    return path

In [11]:
# 运行书上p204案例
if __name__ == '__main__':
    inf = float('inf')
    G = np.array([[inf, 1, 2, inf, inf, inf, inf, inf],
                  [1, inf, inf, 1, inf, inf, inf, inf],
                  [2, inf, inf, 10, 5, inf, inf, inf],
                  [inf, 1, 10, inf, inf, 5, inf, inf],
                  [inf, inf, 5, inf, inf, 10, 1, inf],
                  [inf, inf, inf, 5, 10, inf, inf, 2],
                  [inf, inf, inf, inf, 1, inf, inf, 2],
                  [inf, inf, inf, inf, inf, 2, 2, inf]])
    G_copy = G.copy()
    jidian_set = jidian(G, 8)
    shortest_path, shortest_weight = floyd(G, 8, jidian_set)
    # 求出完全图中的最小权最大匹配
    M = min_match(shortest_weight, len(jidian_set))

In [12]:
    # 求出Euler图的路径矩阵
    G_copy[G_copy < inf] = 1
    G_copy[G_copy == inf] = 0
    C = G_copy.astype(int)
    for m in M:
        path_list = shortest_path[m]
        for i in range(len(path_list) - 1):
            C[path_list[i]][path_list[i + 1]] = 2
            C[path_list[i + 1]][path_list[i]] = 2
    print(euler(C, 6))

[6, 4, 2, 0, 1, 0, 2, 3, 1, 3, 5, 4, 6, 7, 5, 7, 6]
